# Sistema de Equações Lineares (Métodos Diretos)
Noções de métodos diretos de solução numérica para sistemas de equações lineares usando [Julia](http://julialang.org/)

- *Disciplinas*: **EAMB018-A**, **EPET019-A**
- *Professores*: [Adeildo S. Ramos Jr](mailto:adramos@ctec.ufal.br), [Luciana C. L. M. Vieira](mailto:luciana.vieira@ctec.ufal.br)
- *Tutores*: [Leonardo T. Ferreira](mailto:leonardo.ferreira@ctec.ufal.br), [Paulo Victor L. Santos](mailto:paulo.santos@ctec.ufal.br), [Ricardo A. Fernandes](mailto:ricardo.fernandes@ctec.ufal.br)


- Referências:
    - Notas de aula
    - Bezanson, J.; Edelman, A.; Karpinski, S.; Shah V. B. (2017) Julia: A Fresh Approach to Numerical Computing. SIAM Review, 59: 65-98.

## Método da Eliminação de Gauss
A solução usando o Método da Eliminação de Gauss consiste em **duas etapas**:
- Transformação do sistema original em um sistema equivalente usando uma matriz triangular superior
- Resolução do sistema equivalente

### Resolução do Sistema Equivalente
Dado um sistema triangular superior $n$x$n$ com elementos da diagonal de $A$ não nulos, resolve $A x = b$.

In [1]:
using LinearAlgebra

function cn_uppersolver(A::Matrix{Float64}, b::Vector{Float64})    
    n = length(b)
    @assert size(A) == (n, n)
    @assert istriu(A)
    
    x = zeros(n)
    for i = n : -1 : 1
        s = A[i, i+1 : n] ⋅ x[i+1 : n]
        x[i] = (b[i] - s) / A[i, i]
    end
    return x
end;

#### Exemplo

In [2]:
# Matriz A 3x3 triangular superior
A = [3 2 4;
    0 1/3 2/3;
    0 0 -8]

# Vetor b 3x1 de constantes
b = [1, 5/3, 0]

# Criando cópias de A e b
A_, b_ = copy(A), copy(b)

# Resolver sistema equivalente
x = cn_uppersolver(A, b)
println("x: ", x)

# Comparação com solver built-in (\)
x_ = A_ \ b_
println("err: ", x - x_)

x: [-3.0000000000000004, 5.000000000000001, -0.0]
err: [0.0, 0.0, 0.0]


### Escalonamento (Cálculo do Sistema Equivalente)
Percorre os elementos abaixo da diagonal principal, transformando-os, por meio de operações elementares, em zeros, e garantindo que os elementos que já foram transformados anteriormente não sejam mais modificados.

- Operações elementates (não alteram a solução do sistema)
  - Permutar duas equações do sistema
  - Multiplicar uma das equações do sistema por um número real não nulo
  - Somar a uma das equações do sistema uma outra equação desse sistema multiplicada por um número real

In [3]:
function cn_upperfac!(A::Matrix{Float64}, b::Vector{Float64}; pivot=true)  
    n = length(b)
    @assert size(A) == (n, n)
    
    for j = 1 : n-1
        for i = j+1 : n
            
            if pivot; cn_pivot!(A, b, j); end
            
            m = A[i, j] / A[j, j]
            A[i, j] = 0
            
            A[i, j+1:n] -= m * A[j, j+1:n]
            b[i] -= m * b[j]
        end
    end
end;

In [4]:
function cn_pivot!(A, b, j)
    if abs(A[j, j]) < eps()
        _, i_max = findmax(abs.(A[:, j]))
        A[[i_max, j], :] = A[[j, i_max], :]
        b[[i_max, j]] = b[[j, i_max]]
    end
end;

#### Exemplo sem pivoteamento

In [5]:
# Matriz A 4x4 (original, antes do escalonamento)
A = [2.0 1.0 3.0 4.0;
    1.0 4.0 2.0 1.0;
    3.0 2.0 1.0 4.0;
    2.0 2.0 3.0 1.0]

# Vetor b 4x1 (original, antes do escalonamento)
b = [17.0, 9.0, 20.0, 9.0];

# Escalonamento do sistema
cn_upperfac!(A, b, pivot=false)
println("A = ", A); println("b = ", b)

A = [2.0 1.0 3.0 4.0; 0.0 3.5 0.5 -1.0; 0.0 0.0 -3.5714285714285716 -1.8571428571428572; 0.0 0.0 0.0 -2.64]
b = [17.0, 0.5, -5.571428571428571, -7.92]


#### Exemplo com pivoteamento

In [6]:
# Matriz A 4x4 (original, antes do escalonamento)
A = [5.0 10.0 1.0 -2.0;
    4.0 8.0 2.0 -1.0;
    10.0 5.0 3.0 1.0;
    2.0 1.0 1.0 2.0]

# Vetor b 4x1 (original, antes do escalonamento)
b = [-5.0, 3.0, 9.0, 12.0]

# Criando cópias de A e b
A_, b_ = copy(A), copy(b)

# Escalonamento do sistema (SEM pivoteamento)
cn_upperfac!(A_, b_, pivot=false)
println("A = ", A_); println("b = ", b_)

A = [5.0 10.0 1.0 -2.0; 0.0 0.0 1.2 0.6000000000000001; 0.0 0.0 Inf Inf; 0.0 0.0 0.0 NaN]
b = [-5.0, 7.0, Inf, NaN]


In [7]:
# Escalonamento do sistema (COM pivoteamento)
cn_upperfac!(A, b, pivot=true)
println("A = ", A); println("b = ", b)

A = [5.0 10.0 1.0 -2.0; 0.0 -15.0 1.0 5.0; 0.0 0.0 1.2 0.6000000000000001; 0.0 0.0 0.0 1.5999999999999999]
b = [-5.0, 19.0, 7.0, 7.866666666666666]


### Implementação: Método da Eliminação de Gauss

- Evita o cálculo da inversa de $A$
- Consiste em transformar o sistema linear original em um sistema linear equivalente com matriz dos coeficientes triangular superior

In [8]:
function cn_gausselimation(A::Matrix{Float64}, b::Vector{Float64}; pivot=true)
    n = length(b)
    @assert size(A) == (n, n)
    @assert abs(det(A)) > eps()
    
    cn_upperfac!(A, b, pivot=pivot)
    return cn_uppersolver(A, b)
end;

#### Exemplo

In [9]:
# Matriz A 4x4 
A = [0.001 3 -0.25 -2;
    583 0.02 -1 -2.5;
    6 492 -10.3 -2;
    2 3 4 -1]

# Vetor b 4x1
b = [1.23, 0.93, 4.84, 2.2]

# Criando cópias de A e b
A_, b_ = copy(A), copy(b)

# Resolução usando Eliminação de Gauss
x = cn_gausselimation(A, b)
println("x: ", x)

# Comparação com solver built-in (\)
x_ = A_ \ b_
println("err: ", x - x_)

x: [-0.0004980305317037192, 0.015177065304773493, 0.3789649003855466, -0.6396052636062989]
err: [-8.070562447504148e-14, -1.5959455978986625e-15, -1.0624834345662748e-13, 1.0880185641326534e-14]
